In [1]:
import numpy as np
import xtrack as xt         
import xpart as xp
import xdeps as xd   
import inspect
import pickle
import scipy

In [2]:
with open('knob_no_phase.pkl', 'rb') as f:
    knobs = pickle.load(f)

knobs

{'kqtf.b2': 0.0011818757749497125,
 'kqtd.b2': 0.00039995032656042225,
 'ksf.b2': 0.013671217855801818,
 'ksd.b2': -0.027461381805006884,
 'kqt13.l2b2': 0.00044820531861246337,
 'kqt12.l2b2': -0.00325876956245464,
 'kqtl11.l2b2': 0.0036137818922661012,
 'kq10.l2b2': 0.007045139314842348,
 'kq9.l2b2': -0.006972782741462498,
 'kq8.l2b2': 0.0070462830327643845,
 'kq7.l2b2': -0.007842202275448596,
 'kq6.l2b2': 0.0043211262547880455,
 'kq5.l2b2': -0.003337654222225493,
 'kq4.l2b2': 0.001997310268115769,
 'kqx.l2': 0.008779789111959506,
 'kq4.r2b2': -0.00368109021289313,
 'kq5.r2b2': 0.004823885972096867,
 'kq6.r2b2': -0.003955912837622948,
 'kq7.r2b2': 0.007507920296474127,
 'kq8.r2b2': -0.005648259444973232,
 'kq9.r2b2': 0.006765351300046119,
 'kq10.r2b2': -0.005373024477303036,
 'kqtl11.r2b2': 0.00046971444431448617,
 'kqt12.r2b2': -0.0017499689019651523,
 'kqt13.r2b2': 0.004445460352212683,
 'kqt13.l4b2': -0.0034740920274989153,
 'kqt12.l4b2': 0.0023614568708046776,
 'kqtl11.l4b2': -4.10

In [5]:
line_path = './b4_sequence_patched.json'
#line_path = '../HL_IR7_IR3_rematched/b4_sequence_patched.json'
line = xt.Line.from_json(line_path)

Loading line from dict:   0%|          | 0/151124 [00:00<?, ?it/s]

Done loading line from dict.           


In [6]:
line.vars.get_table()

Table: 481044 rows, 2 cols

In [7]:
line.vv['kqtf.b2']

0.0011632091446078041

In [8]:
for k, v in knobs.items():
    line.vars[k] = v

In [9]:
line.vv['kqtf.b2']

0.0011818757749497125

In [10]:
save = True
if save:
    line.to_json('b4_sequence_patched_tune.json')

In [18]:
new_line = xt.Line.from_json('b4_sequence_patched_tune.json')

Loading line from dict:   0%|          | 0/151124 [00:00<?, ?it/s]

Done loading line from dict.           


In [19]:
end_s = new_line.get_length()

TCCS_name = 'tccs.5r3.b2'
TCCP_name = 'tccp.4l3.b2'
TARGET_name = 'target.4l3.b2'
TCLA_name = 'tcla.a5l3.b2'
TCP_name = 'tcp.d6r7.b2'

TCCS_loc = end_s - 6773.7 #6775
TCCP_loc = end_s - 6653.3 #6655
TARGET_loc = end_s - (6653.3 +  0.070/2 + 0.005/2)
TCLA_loc = new_line.get_s_position()[new_line.element_names.index(TCLA_name)]


new_line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name='tccs.5r3.b2')
new_line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccs.5r3.b2_aper')
new_line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name='tccp.4l3.b2')
new_line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccp.4l3.b2_aper')
new_line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name='target.4l3.b2')
new_line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='target.4l3.b2_aper')

In [20]:
tw = new_line.twiss(method='4d')

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [21]:
df = tw.to_pandas()

In [22]:
df0 = df[(df['name']==TCCS_name) |  (df['name']==TCP_name)]# TCP_name]] 
df0

,name,s,x,px,y,py,zeta,delta,ptau,W_matrix,...,bx_chrom,by_chrom,ax_chrom,ay_chrom,wx_chrom,wy_chrom,ddx,ddpx,ddy,ddpy
36187,tcp.d6r7.b2,6459.742798,0.000005,-5.186479e-08,4.866660e-08,3.688225e-09,0.000016,0.0,0.0,"[[19.481277145594657, 4.440892098500626e-16, -...",...,0.763853,75.07037,4.198493,-17.410620,4.267413,77.062897,64.615547,-0.644165,30.173957,0.571265
112161,tccs.5r3.b2,19885.183171,-0.000002,-4.036098e-08,5.168347e-07,-8.297802e-10,0.000016,0.0,0.0,"[[9.06286972844423, 8.881784197001252e-16, 1.2...",...,-72.501641,8.16364,-30.578999,-65.030145,78.686486,65.540558,-17.148928,-0.429061,-9.382788,0.117892


In [23]:
optphase = 158.521582158919/360
optphase

0.4403377282192194

In [24]:
(df0['muy'].iloc[1] - df0['muy'].iloc[0])%1

0.41883876068921566

# Tutorial

In [1]:
import numpy as np
import xtrack as xt

# We build a simple ring
pi = np.pi
lbend = 3
lquad = 0.3
elements = {
    'mqf.1': xt.Quadrupole(length=lquad, k1=0.1),
    'd1.1':  xt.Drift(length=1),
    'mb1.1': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd2.1':  xt.Drift(length=1),

    'mqd.1': xt.Quadrupole(length=lquad, k1=-0.7),
    'd3.1':  xt.Drift(length=1),
    'mb2.1': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd4.1':  xt.Drift(length=1),

    'mqf.2': xt.Quadrupole(length=lquad, k1=0.1),
    'd1.2':  xt.Drift(length=1),
    'mb1.2': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd2.2':  xt.Drift(length=1),

    'mqd.2': xt.Quadrupole(length=lquad, k1=-0.7),
    'd3.2':  xt.Drift(length=1),
    'mb2.2': xt.Bend(length=lbend, k0=pi / 2 / lbend, h=pi / 2 / lbend),
    'd4.2':  xt.Drift(length=1),
}
line = xt.Line(elements=elements, element_names=list(elements.keys()))
line.particle_ref = xt.Particles(p0c=1.2e9, mass0=xt.PROTON_MASS_EV)

In [6]:
line.vars['ciao!'] = 0
line.element_refs['mqf.1'].k1 = line.vars['ciao!'] / lquad

In [14]:
line.vars['ciao!']._get_value()

0

In [12]:
line['mqf.1'].k1 = 3

In [16]:
line['mqf.1'].k1

3.3333333333333335

In [15]:
line.vars['ciao!'] = 1

In [7]:
line.vars['ciao!']._find_dependant_targets()

[vars['ciao!'], element_refs['mqf.1'].k1, element_refs['mqf.1']]

In [17]:
line.vars.get_table()

Table: 2 rows, 2 cols
name     value
t_turn_s     0
ciao!        1

In [20]:
#For each quadrupole we create a variable controlling its integrated strength.
# Expressions can be associated to any beam element property, using the `element_refs`
# attribute of the line. For example:
line.vars['k1l.qf.1'] = 0
line.element_refs['mqf.1'].k1 = line.vars['k1l.qf.1'] / lquad
line.vars['k1l.qd.1'] = 0
line.element_refs['mqd.1'].k1 = line.vars['k1l.qd.1'] / lquad
line.vars['k1l.qf.2'] = 0
line.element_refs['mqf.2'].k1 = line.vars['k1l.qf.2'] / lquad
line.vars['k1l.qd.2'] = 0
line.element_refs['mqd.2'].k1 = line.vars['k1l.qd.2'] / lquad

In [21]:
# When a variable is changed, the corresponding element property is automatically
# updated:
line.vars['k1l.qf.1'] = 0.1
line['mqf.1'].k1 # is 0.333, i.e. 0.1 / lquad


0.33333333333333337

In [22]:
# We can create a variable controlling the integrated strength of the two
# focusing quadrupoles
line.vars['k1lf'] = 0.1
line.vars['k1l.qf.1'] = line.vars['k1lf']
line.vars['k1l.qf.2'] = line.vars['k1lf']
# and a variable controlling the integrated strength of the two defocusing quadrupoles
line.vars['k1ld'] = -0.7
line.vars['k1l.qd.1'] = line.vars['k1ld']
line.vars['k1l.qd.2'] = line.vars['k1ld']


In [23]:
# Changes on the controlling variable are propagated to the two controlled ones and
# to the corresponding element properties:
line.vars['k1lf'] = 0.2
line.vars['k1l.qf.1']._get_value() # is 0.2
line.vars['k1l.qf.2']._get_value() # is 0.2
line['mqf.1'].k1 # is 0.666, i.e. 0.2 / lquad
line['mqf.2'].k1 # is 0.666, i.e. 0.2 / lquad



0.6666666666666667

In [24]:
# The `_info()` method of a variable provides information on the existing relations
# between the variables. For example:
line.vars['k1l.qf.1']._info()

#  vars['k1l.qf.1']._get_value()
   vars['k1l.qf.1'] = 0.2

#  vars['k1l.qf.1']._expr
   vars['k1l.qf.1'] = vars['k1lf']

#  vars['k1l.qf.1']._expr._get_dependencies()
   vars['k1lf'] = 0.2

#  vars['k1l.qf.1']._find_dependant_targets()
   element_refs['mqf.1'].k1



In [25]:
line.vars['k1lf']._info()

#  vars['k1lf']._get_value()
   vars['k1lf'] = 0.2

#  vars['k1lf']._expr is None

#  vars['k1lf']._find_dependant_targets()
   vars['k1l.qf.2']
   element_refs['mqf.2'].k1
   vars['k1l.qf.1']
   element_refs['mqf.1'].k1



In [26]:
line.element_refs['mqf.1'].k1._info()


#  element_refs['mqf.1'].k1._get_value()
   element_refs['mqf.1'].k1 = 0.6666666666666667

#  element_refs['mqf.1'].k1._expr
   element_refs['mqf.1'].k1 = (vars['k1l.qf.1'] / 0.3)

#  element_refs['mqf.1'].k1._expr._get_dependencies()
   vars['k1l.qf.1'] = 0.2

#  element_refs['mqf.1'].k1 does not influence any target



In [27]:
# Expressions can include multiple variables and mathematical operations. For example
line.vars['a'] = 3 * line.functions.sqrt(line.vars['k1lf']) + 2 * line.vars['k1ld']

In [28]:
# As seen above, line.vars['varname'] returns a reference object that
# can be used to build further references, or to inspect its properties.
# To get the current value of the variable, one needs to use `._get_value()`
# For quick access to the current value of a variable, one can use the `line.varval`
# attribute or its shortcut `line.vv`:
line.varval['k1lf'] # is 0.2

0.2

In [29]:
line.vv['k1lf']     # is 0.2

0.2

In [30]:
# Note an important difference when using `line.vars` or `line.varval` in building
# expressions. For example:
line.vars['a'] = 3.
line.vars['b'] = 2 * line.vars['a']


In [31]:
# In this case the reference to the quantity `line.vars['a']` is stored in the
# expression, and the value of `line.vars['b']` is updated when `line.vars['a']`
# changes:
line.vars['a'] = 4.
line.vv['b'] # is 8.


8.0

In [32]:
# On the contrary, when using `line.varval` or `line.vv` in building expressions,
# the current value of the variable is stored in the expression:
line.vv['a'] = 3.
line.vv['b'] = 2 * line.vv['a']
line.vv['b'] # is 6.


6.0

In [33]:
line.vv['a'] = 4.
line.vv['b'] # is still 6.

6.0

In [34]:
# The `line.vars.get_table()` method returns a table with the value of all the
# existing variables:
line.vars.get_table()

Table: 10 rows, 2 cols
name     value
t_turn_s     0
ciao!        1
k1l.qf.1   0.2
k1l.qd.1  -0.7
k1l.qf.2   0.2
k1l.qd.2  -0.7
k1lf       0.2
k1ld      -0.7
a            4
b            6

In [35]:
# Regular expressions can be used to select variables. For example we can select all
# the variables containing `qf` using the following:
var_tab = line.vars.get_table()
var_tab.rows['.*qf.*']

Table: 2 rows, 2 cols
name     value
k1l.qf.1   0.2
k1l.qf.2   0.2